In [1]:
import pandas as pd
import numpy as np
import xarray as xr
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import matplotlib as mpl 
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['savefig.dpi'] = 400
%config InlineBackend.figure_format = 'retina'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings("ignore")

import os
from os import listdir
# os.environ["NUMBA_NUM_THREADS"] = "1"
import random
random.seed(42)
np.random.seed(42)

In [ ]:
dir = os.path.dirname(os.path.abspath('__file__')) # os.path.abspath('')
data_path = os.path.join(dir, '../data/raw')
pro_data_path = os.path.join(data_path, '../processed')
res_path = os.path.join(dir, '../results')
if not os.path.exists(res_path):
    os.makedirs(res_path)
train_path = os.path.join(res_path, "train")
if not os.path.exists(train_path):
    os.makedirs(train_path)

years = [str(y) for y in range(1950, 1981)]  # Match paper's 1950-1980 period
months = [f"{m:02d}" for m in range(1, 13)]  # All 12 months
days = [f"{d:02d}" for d in range(1, 32)]  # 1-31 days
times = ["00:00"]  # Daily data at midnight

# Define region [N, W, S, E]
area = [75, -65, 30, 45]  # Matches the paper's study region

slp_path = os.path.join(data_path, 'slp')
z500_path = os.path.join(data_path, 'z500')

In [ ]:
# Load the data
X = np.load(os.path.join(pro_data_path, "clean_2channel_data.npy"))  # Shape: (time, 2, lat, lon)
X_train = X.copy()
labels = np.load(os.path.join(pro_data_path, "clean_gwl_1950_1980_lbl.npy"))

num_classes = len(set(labels))
print(f"{num_classes} labels:")
print(set(labels))

29 labels:
{'HM', 'HNFA', 'HFZ', 'WW', 'BM', 'TRW', 'SEZ', 'SWZ', 'HNZ', 'NWZ', 'TRM', 'HNA', 'NEA', 'HB', 'SZ', 'WZ', 'SWA', 'HFA', 'SA', 'NEZ', 'NA', 'NWA', 'TM', 'WA', 'NZ', 'SEA', 'WS', 'HNFZ', 'TB'}


In [7]:
# Encode class values
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F

label_encoder = LabelEncoder()
y_indices = label_encoder.fit_transform(labels)
train_idx = np.arange(len(y_indices))
y_indices_train = y_indices[train_idx]
if sum(y_indices_train != y_indices) > 0:
    raise ValueError("Number of labels does not match the number of samples")
np.save(os.path.join(train_path, "train_idx.npy"), train_idx)

lbl_2_idx = {label: i for i, label in zip(range(len(label_encoder.classes_)), label_encoder.classes_)}
idx_2_lbl = {i: label for label, i in lbl_2_idx.items()}
np.save(os.path.join(pro_data_path, "lbl_2_idx.npy"), lbl_2_idx)
np.save(os.path.join(pro_data_path, "idx_2_lbl.npy"), idx_2_lbl)
print("Encoded Class Indices:", y_indices)

# One-hot encode the labels
y_tensor = torch.tensor(y_indices, dtype=torch.long)
y_one_hot = F.one_hot(y_tensor, num_classes=num_classes).float()
y_train = y_one_hot[train_idx,:]
print("One-hot Encoded Labels:", y_one_hot)

Encoded Class Indices: [ 4 13 13 ... 25 25 25]
One-hot Encoded Labels: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [ ]:
valid_time = np.load(os.path.join(pro_data_path, "clean_time_1950_1980.npy"))
valid_time_train = valid_time[train_idx]
valid_time_train = pd.to_datetime(valid_time_train)
valid_time_train.shape

(8946, 2237)

In [ ]:
def compute_mean_variance(X_train, train_path):
    """
    Compute mean and variance for standardization (Z-score normalization).

    :param X_train: Training data of shape (time, 2, lat, lon)
    :return: Mean and variance of shape (2, lat, lon)
    """
    mean_tr = np.mean(X_train, axis=0)  # Compute per-channel mean
    var_tr = np.var(X_train, axis=0)  # Compute per-channel variance
    np.save(os.path.join(train_path, "mean_tr.npy"), mean_tr)
    np.save(os.path.join(train_path, "var_tr.npy"), var_tr)
    return mean_tr, var_tr


def normalize_with_moments(X, mean_tr, var_tr, epsilon=1e-8):
    """
    Apply Z-score normalization to new data.

    :param X: NumPy array of shape (time, 2, lat, lon)
    :param mean_X: Mean of training set (2, lat, lon)
    :param var_X: Variance of training set (2, lat, lon)
    :param epsilon: Small value to prevent division by zero
    :return: Standardized X
    """
    return (X - mean_tr) / np.sqrt(var_tr + epsilon)  # Element-wise normalization

In [ ]:
# Normalization
mean_tr, var_tr = compute_mean_variance(X_train, train_path)

X_train_norm = normalize_with_moments(X_train, mean_tr, var_tr)

In [11]:
def center_seasons(X, valid_month, mean_train_seasonal):
    """
    Perform seasonal adjustment by subtracting the seasonal mean.

    :param X: Data array of shape (time, 2, lat, lon)
    :param valid_month: List of months of timestamps corresponding to each data sample
    :param mean_train_seasonal: Array of seasonal means (12, 2, lat, lon)
    :return: Seasonally adjusted data
    """
    X_adjusted = np.zeros_like(X, dtype=X.dtype)

    season_dict = {
        "Spring": [3, 4, 5],   # Mar, Apr, May
        "Summer": [6, 7, 8],   # Jun, Jul, Aug
        "Fall":   [9, 10, 11],  # Sep, Oct, Nov
        "Winter": [12, 1, 2]  # Dec, Jan, Feb
    }

    for i, months in enumerate(season_dict.values()):
        season_indices = np.isin(valid_month, months)
        X_adjusted[season_indices,:,:,:] = X[season_indices,:,:,:] - mean_train_seasonal[i]

    return X_adjusted

In [ ]:
# Seasonal adjustment
valid_month_train = np.array([t.month for t in valid_time_train])

season_dict = {
        "Spring": [3, 4, 5],   # Mar, Apr, May
        "Summer": [6, 7, 8],   # Jun, Jul, Aug
        "Fall":   [9, 10, 11],  # Sep, Oct, Nov
        "Winter": [12, 1, 2]  # Dec, Jan, Feb
    }

mean_train_seasonal = np.zeros((4, X_train_norm.shape[1], X_train_norm.shape[2], X_train_norm.shape[3]))

for i, months in enumerate(season_dict.values()):
    season_indices = np.isin(valid_month_train, months)
    mean_train_seasonal[i] = np.mean(X_train_norm[season_indices,:,:,:], axis=0)

np.save(os.path.join(train_path, "mean_train_seasonal.npy"), mean_train_seasonal)
train_path = os.path.join(res_path, "train")
X_train_seasonal = center_seasons(X_train_norm, valid_month_train, mean_train_seasonal)
np.save(os.path.join(train_path, "X_train_seasonal.npy"), X_train_seasonal)
np.save(os.path.join(train_path, "y_train.npy"), y_train)

In [ ]:
# Model
import torch.nn as nn

class CirculationTypeClassifier(nn.Module):
    def __init__(self, num_classes, lat, lon, dropout_rate=0.3, out_channels1=8, out_channels2=16, kernel_size=5, fc1_size=50):  # 2 channels (SLP & Z500)
        super(CirculationTypeClassifier, self).__init__()

        self.lat = lat
        self.lon = lon
        padding = kernel_size // 2
      
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=out_channels1, kernel_size=kernel_size, stride=1, padding=padding)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1)
        
        self.conv2 = nn.Conv2d(in_channels=out_channels1, out_channels=out_channels2, kernel_size=kernel_size, stride=1, padding=padding)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(self._calculate_fc_input_size(), fc1_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(fc1_size, num_classes)
        
    def _calculate_fc_input_size(self):
        # Helper function to calculate the input size for the first fully connected layer
        with torch.no_grad():
            x = torch.zeros(1, 2, self.lat, self.lon)
            x = self.pool1(F.relu(self.conv1(x)))
            x = self.pool2(F.relu(self.conv2(x)))
            x = self.flatten(x)
            return x.shape[1]
    
    def forward(self, x):
        # Forward pass
        x = F.relu(self.conv1(x))  # Assuming ReLU activation
        x = self.pool1(x)
        x = F.relu(self.conv2(x))  # Assuming ReLU activation
        x = self.pool2(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))  # Assuming ReLU activation
        x = self.dropout(x)
        x = self.fc2(x)  # Assuming no activation function for the output layer
        return x

In [ ]:
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import balanced_accuracy_score
import optuna
import json

# Define cross-validation strategy
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set ensemble number (30 models as paper)
ensemble_size = 30
ensemble_per_fold = ensemble_size // outer_cv.get_n_splits()

# Data dimensions
time_dim = X.shape[0]
n_vars = X.shape[1]
lat = X.shape[2]
lon = X.shape[3]

epochs = 35 # 35 as paper
patience = 5 # 5 as paper
batch_size = 128 # 128 as paper
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = os.path.join(res_path, "models")
if not os.path.exists(model_path):
    os.makedirs(model_path)

best_models = []  # Store best models for ensemble


X_train_seasonal = torch.tensor(X_train_seasonal, dtype=torch.float32)  # Ensure X is a PyTorch tensor
y_indices_train = np.array(y_indices_train)
y_train = torch.tensor(y_train, dtype=torch.float32)

all_test_preds = []
all_test_trues = []

all_train_out = []
all_test_out = []

for fold_idx, (train_idx_out, test_idx_out) in enumerate(outer_cv.split(X_train_seasonal, y_indices_train)):
    X_train_out, X_test_out = X_train_seasonal[train_idx_out], X_train_seasonal[test_idx_out]
    y_train_out, y_test_out = y_train[train_idx_out], y_train[test_idx_out]
    y_indices_test_out = y_indices_train[test_idx_out]

    all_train_out.extend(train_idx_out.tolist())
    all_test_out.extend(test_idx_out.tolist())

    fold_ensemble_preds = []
    for esb_idx in range(ensemble_per_fold):
        torch.manual_seed(esb_idx)
        np.random.seed(esb_idx)
        random.seed(esb_idx)

        def objective(trial):
            lr = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
            wd = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
            dr = trial.suggest_float("dropout", 0.2, 0.6)
            out_c1 = trial.suggest_categorical("out_channels1", [4, 8, 16])
            out_c2 = trial.suggest_categorical("out_channels2", [8, 16, 32])
            ks = trial.suggest_categorical("kernel_size", [3, 5, 7])
            fc1 = trial.suggest_categorical("fc1_size", [32, 64, 128])

            inner_scores = []
            for train_idx_in, val_idx_in in inner_cv.split(X_train_out, y_train_out.argmax(dim=1)):
                X_train_fold = X_train_out[train_idx_in]
                y_train_fold = y_train_out[train_idx_in]
                X_val_fold = X_train_out[val_idx_in]
                y_val_fold = y_train_out[val_idx_in]

                model = CirculationTypeClassifier(num_classes, lat, lon, dropout_rate=dr, out_channels1=out_c1,
                                                  out_channels2=out_c2, kernel_size=ks, fc1_size=fc1).to(device)
                optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
                criterion = nn.CrossEntropyLoss()

                train_loader = DataLoader(TensorDataset(X_train_fold, y_train_fold), batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(TensorDataset(X_val_fold, y_val_fold), batch_size=batch_size, shuffle=False)

                best_val_score = -np.inf
                p_counter = 0

                for epoch in range(epochs):
                    _= model.train()
                    for Xb, yb in train_loader:
                        Xb, yb = Xb.to(device), yb.to(device)
                        optimizer.zero_grad()
                        output = model(Xb)
                        loss = criterion(output, yb.argmax(dim=1))
                        _= loss.backward()
                        _= optimizer.step()

                    _= model.eval()
                    y_pred, y_true = [], []
                    with torch.no_grad():
                        for Xb, yb in val_loader:
                            Xb = Xb.to(device)
                            output = model(Xb)
                            y_pred.extend(torch.argmax(output, dim=1).cpu().numpy())
                            y_true.extend(yb.argmax(dim=1).cpu().numpy())
                    val_score = balanced_accuracy_score(y_true, y_pred)
                    if val_score > best_val_score:
                        best_val_score = val_score
                        p_counter = 0
                    else:
                        p_counter += 1
                        if p_counter >= patience:
                            break

                inner_scores.append(best_val_score)
            return np.mean(inner_scores)

        study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=esb_idx))
        study.optimize(objective, n_trials=20)
        best_hyperparams = study.best_params

        # Save best model with optimal hyperparameters
        best_model = CirculationTypeClassifier(num_classes, lat, lon,
                                               dropout_rate=best_hyperparams['dropout'],
                                               out_channels1=best_hyperparams['out_channels1'],
                                               out_channels2=best_hyperparams['out_channels2'],
                                               kernel_size=best_hyperparams['kernel_size'],
                                               fc1_size=best_hyperparams['fc1_size']).to(device)

        optimizer = optim.Adam(best_model.parameters(),
                               lr=best_hyperparams['learning_rate'],
                               weight_decay=best_hyperparams['weight_decay'])
        
        criterion = nn.CrossEntropyLoss()
        train_loader = DataLoader(TensorDataset(X_train_out, y_train_out), batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            _= best_model.train()
            for Xb, yb in train_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                optimizer.zero_grad()
                output = best_model(Xb)
                loss = criterion(output, yb.argmax(dim=1))
                _= loss.backward()
                _= optimizer.step()

        m_path = os.path.join(model_path, f"model_f{fold_idx}_e{esb_idx}.pt")
        torch.save(best_model.state_dict(), m_path)
        with open(os.path.join(model_path, f"params_f{fold_idx}_e{esb_idx}.json"), "w") as f:
            json.dump(best_hyperparams, f, indent=2)

        _= best_model.eval()
        y_pred = []
        with torch.no_grad():
            for Xb, yb in DataLoader(TensorDataset(X_test_out, y_test_out), batch_size=batch_size):
                Xb = Xb.to(device)
                outputs = best_model(Xb)
                y_pred.extend(torch.argmax(outputs, dim=1).cpu().numpy())

        fold_ensemble_preds.append(y_pred)

    fold_ensemble_preds = np.array(fold_ensemble_preds)
    assert fold_ensemble_preds.shape == (ensemble_per_fold, X_test_out.shape[0]), "fold_ensemble_preds must have correct dim!"
    fold_ensemble_preds = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=fold_ensemble_preds)

    all_test_preds.extend(fold_ensemble_preds.tolist())
    all_test_trues.extend(y_indices_test_out.tolist())

np.save(os.path.join(train_path, "cvout_test_preds.npy"), np.array(all_test_preds))
np.save(os.path.join(train_path, "cvout_test_trues.npy"), np.array(all_test_trues))
np.save(os.path.join(train_path, f"train_idx_out.npy"), all_train_out)
np.save(os.path.join(train_path, f"test_idx_out.npy"), all_test_out)